# Gold Layer - Create Business-Ready Tables
This notebook creates Gold tables with business transformations using the updated Silver schema.

In [ ]:
# Create daily reports Gold table
%sql
CREATE TABLE IF NOT EXISTS gold.taxi.daily_reports (
  pickup_date DATE COMMENT 'Trip pickup date',
  
  -- Volume & Revenue metrics
  num_trips BIGINT COMMENT 'Total number of trips',
  total_passengers BIGINT COMMENT 'Total number of passengers',
  total_revenue DECIMAL(15, 2) COMMENT 'Total revenue amount',
  total_fare DECIMAL(15, 2) COMMENT 'Total fare amount',
  total_tips DECIMAL(15, 2) COMMENT 'Total tip amount',
  total_tolls DECIMAL(15, 2) COMMENT 'Total tolls amount',
  total_extra DECIMAL(15, 2) COMMENT 'Total extra charges',
  avg_total_amount DECIMAL(10, 2) COMMENT 'Average total amount per trip',
  median_total_amount DECIMAL(10, 2) COMMENT 'Median total amount per trip',
  
  -- Vendor metrics
  num_trips_cmt BIGINT COMMENT 'Number of trips by Creative Mobile Technologies',
  num_trips_curb BIGINT COMMENT 'Number of trips by Curb Mobility',
  num_trips_myle BIGINT COMMENT 'Number of trips by Myle Technologies',
  num_trips_helix BIGINT COMMENT 'Number of trips by Helix',
  total_revenue_cmt DECIMAL(15, 2) COMMENT 'Total revenue by Creative Mobile Technologies',
  total_revenue_curb DECIMAL(15, 2) COMMENT 'Total revenue by Curb Mobility',
  total_revenue_myle DECIMAL(15, 2) COMMENT 'Total revenue by Myle Technologies',
  total_revenue_helix DECIMAL(15, 2) COMMENT 'Total revenue by Helix',
  
  -- Location metrics
  distinct_pickup_locations BIGINT COMMENT 'Number of distinct pickup locations',
  distinct_dropoff_locations BIGINT COMMENT 'Number of distinct dropoff locations',
  
  -- Efficiency metrics
  avg_trip_distance_km DECIMAL(10, 2) COMMENT 'Average trip distance in km',
  avg_trip_duration_min DECIMAL(10, 2) COMMENT 'Average trip duration in minutes',
  avg_speed_kmph DECIMAL(10, 2) COMMENT 'Average speed in km/h',
  revenue_per_minute DECIMAL(10, 4) COMMENT 'Revenue per minute',
  revenue_per_km DECIMAL(10, 2) COMMENT 'Revenue per kilometer',
  
  -- Service quality metrics
  avg_tip_pct DECIMAL(5, 4) COMMENT 'Average tip percentage',
  pct_trips_with_tip DECIMAL(5, 4) COMMENT 'Percentage of trips with tips',
  
  -- Trip distance categories
  short_trips BIGINT COMMENT 'Number of trips <= 1km',
  long_trips BIGINT COMMENT 'Number of trips >= 10km',
  
  -- Payment methods
  payment_methods_used BIGINT COMMENT 'Number of distinct payment methods used',
  num_credit_card BIGINT COMMENT 'Number of credit card payments',
  num_cash BIGINT COMMENT 'Number of cash payments',
  num_no_charge BIGINT COMMENT 'Number of no charge trips',
  num_dispute BIGINT COMMENT 'Number of disputed payments',
  num_unknown BIGINT COMMENT 'Number of unknown payment types',
  num_voided BIGINT COMMENT 'Number of voided trips',
  
  -- Data quality metrics
  zero_passenger_trips BIGINT COMMENT 'Number of trips with zero passengers',
  invalid_total_trips BIGINT COMMENT 'Number of trips with invalid totals',
  
  -- Metadata
  taxi_type STRING COMMENT 'Type of taxi (yellow/green)',
  created_at TIMESTAMP COMMENT 'Record creation timestamp'
)
USING DELTA
PARTITIONED BY (taxi_type)
COMMENT 'Daily aggregated taxi trip reports'
TBLPROPERTIES (
  'delta.autoOptimize.optimizeWrite' = 'true',
  'delta.autoOptimize.autoCompact' = 'true'
);

In [ ]:
# Function to create daily reports for Yellow Taxi (updated for new Silver schema)
def create_yellow_taxi_daily_reports():
    query = """
    INSERT INTO gold.taxi.daily_reports
    WITH yellow_base AS (
        SELECT
            DATE(pickup_datetime) AS pickup_date,
            trip_duration_minutes,
            total_amount,
            fare_amount,
            tip_amount,
            tolls_amount,
            improvement_surcharge,
            extra,
            trip_distance_km,
            passenger_count,
            vendor_name,
            pickup_location_id,
            dropoff_location_id,
            payment_type_id
        FROM silver.taxi.yellow_taxi
    ),
    aggregated AS (
        SELECT
            pickup_date,
            -- Volume & revenue
            COUNT(*) AS num_trips,
            SUM(passenger_count) AS total_passengers,
            SUM(total_amount) AS total_revenue,
            SUM(fare_amount) AS total_fare,
            SUM(tip_amount) AS total_tips,
            SUM(tolls_amount) AS total_tolls,
            SUM(extra) AS total_extra,
            AVG(total_amount) AS avg_total_amount,
            PERCENTILE(total_amount, 0.5) AS median_total_amount,
            
            -- Vendor metrics
            SUM(CASE WHEN vendor_name = 'Creative Mobile Technologies, LLC' THEN 1 ELSE 0 END) AS num_trips_cmt,
            SUM(CASE WHEN vendor_name = 'Curb Mobility, LLC' THEN 1 ELSE 0 END) AS num_trips_curb,
            SUM(CASE WHEN vendor_name = 'Myle Technologies Inc' THEN 1 ELSE 0 END) AS num_trips_myle,
            SUM(CASE WHEN vendor_name = 'Helix' THEN 1 ELSE 0 END) AS num_trips_helix,
            SUM(CASE WHEN vendor_name = 'Creative Mobile Technologies, LLC' THEN total_amount ELSE 0 END) AS total_revenue_cmt,
            SUM(CASE WHEN vendor_name = 'Curb Mobility, LLC' THEN total_amount ELSE 0 END) AS total_revenue_curb,
            SUM(CASE WHEN vendor_name = 'Myle Technologies Inc' THEN total_amount ELSE 0 END) AS total_revenue_myle,
            SUM(CASE WHEN vendor_name = 'Helix' THEN total_amount ELSE 0 END) AS total_revenue_helix,
            
            -- Location metrics
            COUNT(DISTINCT pickup_location_id) AS distinct_pickup_locations,
            COUNT(DISTINCT dropoff_location_id) AS distinct_dropoff_locations,
            
            -- Efficiency metrics (already in km from Silver)
            AVG(trip_distance_km) AS avg_trip_distance_km,
            AVG(trip_duration_minutes) AS avg_trip_duration_min,
            SUM(trip_distance_km) / NULLIF(SUM(trip_duration_minutes), 0) * 60 AS avg_speed_kmph,
            SUM(total_amount) / NULLIF(SUM(trip_duration_minutes), 0) AS revenue_per_minute,
            SUM(total_amount) / NULLIF(SUM(trip_distance_km), 0) AS revenue_per_km,
            
            -- Service quality metrics
            AVG(CASE WHEN total_amount > 0 THEN tip_amount / total_amount ELSE NULL END) AS avg_tip_pct,
            COUNT(CASE WHEN tip_amount > 0 THEN 1 END) * 1.0 / COUNT(*) AS pct_trips_with_tip,
            
            -- Trip distance categories (km-based)
            COUNT(CASE WHEN trip_distance_km <= 1.6 THEN 1 END) AS short_trips, -- <= 1 mile in km
            COUNT(CASE WHEN trip_distance_km >= 16.1 THEN 1 END) AS long_trips, -- >= 10 miles in km
            
            -- Payment methods
            COUNT(DISTINCT payment_type_id) AS payment_methods_used,
            COUNT(CASE WHEN payment_type_id = 1 THEN 1 END) AS num_credit_card,
            COUNT(CASE WHEN payment_type_id = 2 THEN 1 END) AS num_cash,
            COUNT(CASE WHEN payment_type_id = 3 THEN 1 END) AS num_no_charge,
            COUNT(CASE WHEN payment_type_id = 4 THEN 1 END) AS num_dispute,
            COUNT(CASE WHEN payment_type_id = 5 THEN 1 END) AS num_unknown,
            COUNT(CASE WHEN payment_type_id = 6 THEN 1 END) AS num_voided,
            
            -- Data quality metrics
            COUNT(CASE WHEN passenger_count = 0 THEN 1 END) AS zero_passenger_trips,
            COUNT(CASE WHEN total_amount <= 0 THEN 1 END) AS invalid_total_trips
        FROM yellow_base
        GROUP BY pickup_date
    )
    SELECT 
        pickup_date,
        num_trips, total_passengers, total_revenue, total_fare, total_tips,
        total_tolls, total_extra, avg_total_amount, median_total_amount,
        num_trips_cmt, num_trips_curb, num_trips_myle, num_trips_helix,
        total_revenue_cmt, total_revenue_curb, total_revenue_myle, total_revenue_helix,
        distinct_pickup_locations, distinct_dropoff_locations,
        avg_trip_distance_km, avg_trip_duration_min, avg_speed_kmph,
        revenue_per_minute, revenue_per_km,
        avg_tip_pct, pct_trips_with_tip,
        short_trips, long_trips,
        payment_methods_used, num_credit_card, num_cash, num_no_charge,
        num_dispute, num_unknown, num_voided,
        zero_passenger_trips, invalid_total_trips,
        'yellow' as taxi_type,
        current_timestamp() as created_at
    FROM aggregated
    ORDER BY pickup_date
    """
    spark.sql(query)

# Execute Yellow Taxi daily reports creation
create_yellow_taxi_daily_reports()
print("Yellow Taxi daily reports created successfully with updated Silver schema")

In [ ]:
# Function to create daily reports for Green Taxi (updated for new Silver schema)
def create_green_taxi_daily_reports():
    query = """
    INSERT INTO gold.taxi.daily_reports
    WITH green_base AS (
        SELECT
            DATE(pickup_datetime) AS pickup_date,
            trip_duration_minutes,
            total_amount,
            fare_amount,
            tip_amount,
            tolls_amount,
            improvement_surcharge,
            extra,
            trip_distance_km,
            passenger_count,
            vendor_name,
            pickup_location_id,
            dropoff_location_id,
            payment_type_id
        FROM silver.taxi.green_taxi
    ),
    aggregated AS (
        SELECT
            pickup_date,
            -- Volume & revenue
            COUNT(*) AS num_trips,
            SUM(passenger_count) AS total_passengers,
            SUM(total_amount) AS total_revenue,
            SUM(fare_amount) AS total_fare,
            SUM(tip_amount) AS total_tips,
            SUM(tolls_amount) AS total_tolls,
            SUM(extra) AS total_extra,
            AVG(total_amount) AS avg_total_amount,
            PERCENTILE(total_amount, 0.5) AS median_total_amount,
            
            -- Vendor metrics
            SUM(CASE WHEN vendor_name = 'Creative Mobile Technologies, LLC' THEN 1 ELSE 0 END) AS num_trips_cmt,
            SUM(CASE WHEN vendor_name = 'Curb Mobility, LLC' THEN 1 ELSE 0 END) AS num_trips_curb,
            SUM(CASE WHEN vendor_name = 'Myle Technologies Inc' THEN 1 ELSE 0 END) AS num_trips_myle,
            0 AS num_trips_helix, -- Green taxi doesn't have Helix vendor
            SUM(CASE WHEN vendor_name = 'Creative Mobile Technologies, LLC' THEN total_amount ELSE 0 END) AS total_revenue_cmt,
            SUM(CASE WHEN vendor_name = 'Curb Mobility, LLC' THEN total_amount ELSE 0 END) AS total_revenue_curb,
            SUM(CASE WHEN vendor_name = 'Myle Technologies Inc' THEN total_amount ELSE 0 END) AS total_revenue_myle,
            0 AS total_revenue_helix, -- Green taxi doesn't have Helix vendor
            
            -- Location metrics
            COUNT(DISTINCT pickup_location_id) AS distinct_pickup_locations,
            COUNT(DISTINCT dropoff_location_id) AS distinct_dropoff_locations,
            
            -- Efficiency metrics (already in km from Silver)
            AVG(trip_distance_km) AS avg_trip_distance_km,
            AVG(trip_duration_minutes) AS avg_trip_duration_min,
            SUM(trip_distance_km) / NULLIF(SUM(trip_duration_minutes), 0) * 60 AS avg_speed_kmph,
            SUM(total_amount) / NULLIF(SUM(trip_duration_minutes), 0) AS revenue_per_minute,
            SUM(total_amount) / NULLIF(SUM(trip_distance_km), 0) AS revenue_per_km,
            
            -- Service quality metrics
            AVG(CASE WHEN total_amount > 0 THEN tip_amount / total_amount ELSE NULL END) AS avg_tip_pct,
            COUNT(CASE WHEN tip_amount > 0 THEN 1 END) * 1.0 / COUNT(*) AS pct_trips_with_tip,
            
            -- Trip distance categories (km-based)
            COUNT(CASE WHEN trip_distance_km <= 1.6 THEN 1 END) AS short_trips, -- <= 1 mile in km
            COUNT(CASE WHEN trip_distance_km >= 16.1 THEN 1 END) AS long_trips, -- >= 10 miles in km
            
            -- Payment methods
            COUNT(DISTINCT payment_type_id) AS payment_methods_used,
            COUNT(CASE WHEN payment_type_id = 1 THEN 1 END) AS num_credit_card,
            COUNT(CASE WHEN payment_type_id = 2 THEN 1 END) AS num_cash,
            COUNT(CASE WHEN payment_type_id = 3 THEN 1 END) AS num_no_charge,
            COUNT(CASE WHEN payment_type_id = 4 THEN 1 END) AS num_dispute,
            COUNT(CASE WHEN payment_type_id = 5 THEN 1 END) AS num_unknown,
            COUNT(CASE WHEN payment_type_id = 6 THEN 1 END) AS num_voided,
            
            -- Data quality metrics
            COUNT(CASE WHEN passenger_count = 0 THEN 1 END) AS zero_passenger_trips,
            COUNT(CASE WHEN total_amount <= 0 THEN 1 END) AS invalid_total_trips
        FROM green_base
        GROUP BY pickup_date
    )
    SELECT 
        pickup_date,
        num_trips, total_passengers, total_revenue, total_fare, total_tips,
        total_tolls, total_extra, avg_total_amount, median_total_amount,
        num_trips_cmt, num_trips_curb, num_trips_myle, num_trips_helix,
        total_revenue_cmt, total_revenue_curb, total_revenue_myle, total_revenue_helix,
        distinct_pickup_locations, distinct_dropoff_locations,
        avg_trip_distance_km, avg_trip_duration_min, avg_speed_kmph,
        revenue_per_minute, revenue_per_km,
        avg_tip_pct, pct_trips_with_tip,
        short_trips, long_trips,
        payment_methods_used, num_credit_card, num_cash, num_no_charge,
        num_dispute, num_unknown, num_voided,
        zero_passenger_trips, invalid_total_trips,
        'green' as taxi_type,
        current_timestamp() as created_at
    FROM aggregated
    ORDER BY pickup_date
    """
    spark.sql(query)

# Execute Green Taxi daily reports creation
create_green_taxi_daily_reports()
print("Green Taxi daily reports created successfully with updated Silver schema")

In [ ]:
# Verify Gold data
%sql
SELECT 
  taxi_type,
  COUNT(*) as days_count,
  MIN(pickup_date) as min_date,
  MAX(pickup_date) as max_date,
  SUM(num_trips) as total_trips,
  SUM(total_revenue) as total_revenue
FROM gold.taxi.daily_reports 
GROUP BY taxi_type
ORDER BY taxi_type;

In [ ]:
# Sample of Gold data
%sql
SELECT 
  pickup_date,
  taxi_type,
  num_trips,
  total_revenue,
  avg_total_amount,
  avg_trip_distance_km,
  avg_tip_pct
FROM gold.taxi.daily_reports 
WHERE pickup_date BETWEEN '2020-01-01' AND '2020-01-10'
ORDER BY pickup_date, taxi_type;

In [ ]:
# Optimize Gold table
%sql
OPTIMIZE gold.taxi.daily_reports;

In [ ]:
# Optimize Gold table
%sql
OPTIMIZE gold.taxi.daily_reports;